In [16]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.activations import linear, relu, sigmoid

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
pd.options.mode.chained_assignment = None
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s4e3/sample_submission.csv
/kaggle/input/playground-series-s4e3/train.csv
/kaggle/input/playground-series-s4e3/test.csv


# **Input data & Scoring features**

Use top 10 features for model training

In [ ]:
train_df = pd.read_csv('/kaggle/input/playground-series-s4e3/train.csv')

train_df.head()


In [ ]:
X_feat = train_df.iloc[:,1:-7].abs()
y_train = train_df.iloc[:,-7:]

#X_feat.head()
y_train.head()

In [ ]:
BestFeatures = SelectKBest(score_func=chi2, k=10)

fit = BestFeatures.fit(X_feat,y_train)

df_scores = pd.DataFrame(fit.scores_)
df_columns = pd.DataFrame(X_feat.columns)

f_Scores = pd.concat([df_columns,df_scores],axis=1)
f_Scores.columns = ['Specs','Score']  
f_Scores.sort_values(by=['Score'])
#f_Scores
features = f_Scores.nlargest(10,'Score')
print(features)
features = features['Specs'].tolist()


# **Scaling of features**

In [ ]:
ss = StandardScaler()
X_scaled = ss.fit_transform(X_feat[features])
#X_train.head()

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_train, test_size=0.2, random_state=69)

# **Building of Multi-label Neural Network using Tensorflow**

In [ ]:
tf.random.set_seed(69)
model = Sequential([
    tf.keras.Input(shape=(10,)),
    Dense(25, activation='relu', name='L1'),
    Dense(10, activation='relu', name='L2'),
    Dense(7, activation='sigmoid', name='L3')
], name='model'
)
model.summary()

In [ ]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(
    X_train,y_train,
    epochs=40,
    validation_data=(X_test, y_test)
)

In [ ]:
pred = pd.read_csv('/kaggle/input/playground-series-s4e3/test.csv')

pred_scaled = ss.fit_transform(pred[features])
pred_scaled



In [ ]:
result = model.predict(pred_scaled)

In [ ]:
submission = pd.DataFrame(result, columns = y_train.columns)
submission['id'] = pred.id


submission.to_csv('submission.csv', index=False)

submission.head()